In [1]:
from nltk.probability import FreqDist
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import heapq
import nltk.data
import csv
import numpy as np
import pandas as pd
import math
nltk.download('stopwords')
nltk.download('punkt')



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ines\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ines\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
#read the lyrics.csv
# the lyrics and scores are empty for new songs, they have to be filled first
df = pd.read_csv('Lyrics.csv',delimiter=";",encoding='latin1')
print(df)

               Song Title                                          Artist  \
0           Feel So Close                                   Calvin Harris   
1       Turn Up the Music                                     Chris Brown   
2    Club Can't Handle Me                          Flo Rida, David Guetta   
3           Rain in Ibiza  Felix Jaehn, The Stickmen Project, Calum Scott   
4                Believer                                 Imagine Dragons   
..                    ...                                             ...   
989                   NaN                                             NaN   
990                   NaN                                             NaN   
991                   NaN                                             NaN   
992                   NaN                                             NaN   
993                   NaN                                             NaN   

                                                Lyrics    bpm  \
0    I fee

In [8]:
# API Call for lyrics. 
# Don't always run it for all songs, as the API only allows 100 calls per day. 
# Therefore put i on the first line with empty lyrics
i = 0
from lyricsgenius import Genius
token = 'Wbdl4zQaphgb0HpfLCqJuhJWNU2B2m7NDXTh3oN9TE5bmM77K1fS68czEbS_wSHy'
genius = Genius(token)
songs =[]
for artist in df.loc[:,"Artist"]:
    if i>103: #put here the first empty line
        print('Start')
        while True:
            try:
                song = genius.search_song(df.loc[i]['Song Title'],artist)
                break
            except:
                pass
        df.at[i,'Lyrics'] = song.lyrics
    i+= 1
#print(artists[1].lyrics)   

Start
Searching for "God is a woman" by Ariana Grande...
Done.
Start
Searching for "break up with your girlfriend, i'm bored" by Ariana Grande...
Done.
Start
Searching for "Side To Side" by Ariana Grande...
Done.
Start
Searching for "Pump It" by Black Eyed Peas...
Done.
Start
Searching for "Boom Boom Pow" by Black Eyed Peas...
Done.
Start
Searching for "Stronger (What Doesn't Kill You)" by Kelly Clarkson...
Done.
Start
Searching for "Stronger" by Kanye West...
Done.
Start
Searching for "CUFF IT" by Beyoncé...
Done.
Start
Searching for "Yeah!" by Usher, Lil Jon, Ludacris...
Done.
Start
Searching for "Don't stop the music" by Rihanna...
Done.
Start
Searching for "Run the World (Girls)" by Beyoncé...
Done.
Start
Searching for "Rock Your Body" by Justin Timberlake...
Done.
Start
Searching for "BREAK MY SOUL" by Beyoncé...
Done.
Start
Searching for "Survivor" by Destiny's Child...
Done.
Start
Searching for "Lose Yourself" by Eminem...
Done.
Start
Searching for "Try again" by Aaliyah...
Done

Done.
Start
Searching for "NUMB" by Hayden James, GRAACE...
Done.
Start
Searching for "Better Than Ever" by Flight Facilities, Aloe Blacc...
Done.
Start
Searching for "Love to Go" by Lost Frequencies, Zonderling, Kelvin Jones...
Done.
Start
Searching for "Praise You - Radio Edit" by Fatboy Slim...
No results found for: 'Praise You - Radio Edit Fatboy Slim'


AttributeError: 'NoneType' object has no attribute 'lyrics'

In [10]:
# Here the new lyrics are written to the file more_lyric.csv. 
# Either change this to write to lyrics.csv or copy-paste. 
# This was only the test phase so far.
df.to_csv('more_lyrics.csv',sep=';',encoding='utf-8-sig')

In [8]:
#lookup line for finding errors
print(df.iloc[103]['Song Title'])

It Was A Very Good Year


In [12]:
#scores for our categories
#TODO
push_myself ={'push','strive','grind','plod','labor','moil','strain','tug','speed','rocket','opportunities','opportunity','hard','shouting','stopping','force','pain','believer','limit','sky','shit','fuck','unstoppable','pump','adrenaline'}
have_fun = {'fun','happy','fancy','savor','enjoy','good','great','awesome','amazing','nice','sweet','fantastic','sunshine','dancing','alive','dance','party','dj','music','sex','whoop','tickle','clowns'}
stress_relief = {'stress', 'relief','stressless','stressfree','reduction','alright','kind','sleep','breathe','breathing','time','love','kiss','sun','stars','moon','sky'}
relax_focus={'relax','focus','focused','focusing','focussed','focussing','relaxation','recreation','relaxing','relaxed','recreational','unwind','unwinding','de-stressing','soothe','soothing','eased','ease','dream','fairy','paradise','beautiful','mind','butterflies','concentrate','concentration','chill','chilling','chillin','chilled'}
get_motivated ={'motivation','motivating','motivated','motivate','motifing','animate','animating','animated','encouraging','encourage','encouraged','exciting','inspiring','inspire','inspired','inspirational','stirring','exhilarating','uplifting','winning','win','winner','easy','survive','tiger','work','countdown'}

In [13]:
#counters for statistics
push_myself_counter= 0
have_fun_counter = 0
stress_relief_counter = 0
relax_focus_counter = 0
get_motivated_counter = 0

In [14]:

def summerise(input, i):
    scores = [0,0,0,0,0] #push myself, have fun, stress relief, relax/focus, motivation
    
    # (1) Convert paragraphs to sentences
    sentence_detector = nltk.data.load('tokenizers/punkt/english.pickle')
    sentences = sentence_detector.tokenize(input)
    #sentences = nltk.sent_tokenize(input)
    
    # (2) Tokenisation: split sentences to words
    tokenizer = RegexpTokenizer('\w+')
    
    # (3) Preprocessing: lowercase words and remove stopwords
    base_words = [word.lower() for word in tokenizer.tokenize(input)]
    words_fist_step = [word for word in base_words if word not in stopwords.words()]
    
    #TODO
    #processing because of problems with API lyrics. Have a look at them, the problem are the words in [], they distort the statistics
    #This will filter out some common ones, but it is best to use a filter that excludes all words surrounded by square brackets.
    words = [word for word in words_fist_step if word not in ['chorus','lyrics','lyric','guitar solo','solo','verse','verse 1','verse 2','verse 3','verse 4','verse 5','outro','intro','refrain','instrumental','instrumental post-chorus','interlude','bridge','pre chorus','pre-chorus','post-chorus','post chorus','ad lib','ad-lib']]

    # (4) Find Weighted Frequency of Occurence      
    word_frequencies = FreqDist(words)
        
    maximum_frequency = max(word_frequencies.values())

    for word in word_frequencies.keys():
        word_frequencies[word] = (word_frequencies[word]/maximum_frequency)
    #print('---------------------------------------------')
    print(word_frequencies.most_common())
    #print('---------------------------------------------')
    #print(word_frequencies.keys())
    for word in word_frequencies.keys():
        if(word in push_myself):
            scores[0]+= 1
        if(word in have_fun):
            scores[1]+= 1    
        if(word in stress_relief):
            scores[2]+= 1    
        if(word in relax_focus):
            scores[3]+= 1   
        if(word in get_motivated):
            scores[4]+= 1   
   # print(summary)
    if scores[0]>0:
        global push_myself_counter
        push_myself_counter+= 1
    if scores[1]>0:
        global have_fun_counter
        have_fun_counter+= 1
    if scores[2]>0:
        global stress_relief_counter
        stress_relief_counter+= 1
    if scores[3]>0:
        global relax_focus_counter
        relax_focus_counter+= 1
    if scores[4]>0:
        global get_motivated_counter
        get_motivated_counter+= 1
    print('Push myself score:'+ str(scores[0]))
    print('Have fun score:'+ str(scores[1]))
    print('Stress relief score:'+ str(scores[2]))
    print('Relax / focus score:'+ str(scores[3]))
    print('Get motivated score:'+ str(scores[4]))
    df.at[i,'Push myself score'] = scores[0]
    df.at[i,'Have fun score'] = scores[1]
    df.at[i,'Stress relief score'] = scores[2]
    df.at[i,'Relax / focus score'] = scores[3]
    df.at[i,'Get motivated score'] = scores[4]
    return scores


In [15]:
#Here the processing happens, the scores re calculated, if the are empty so far
i = 0
for song in df.loc[:,"Lyrics"]:
    print ('******** '+df.loc[i]['Song Title']+' ********')
    if math.isnan(df.loc[i]['Get motivated score']):
        summerise(song, i)
    i+= 1
    #print('*******************************************************')
    print('\n')

******** Feel So Close ********
[('feel', 1.0), ('close', 1.0), ('stopping', 1.0), ('force', 0.4), ('field', 0.4), ('wear', 0.4), ('heart', 0.4), ('sleeve', 0.4), ('big', 0.4), ('deal', 0.4), ('love', 0.4), ('pours', 0.4), ('surrounds', 0.4), ('waterfall', 0.4)]
Push myself score:2
Have fun score:0
Stress relief score:1
Relax / focus score:0
Get motivated score:0


******** Turn Up the Music ********
[('turn', 1.0), ('music', 0.49056603773584906), ('whoa', 0.4716981132075472), ('hands', 0.33962264150943394), ('dance', 0.33962264150943394), ('put', 0.22641509433962265), ('air', 0.16981132075471697), ('girl', 0.1320754716981132), ('louder', 0.1320754716981132), ('life', 0.1320754716981132), ('sexy', 0.07547169811320754), ('baby', 0.05660377358490566), ('stop', 0.05660377358490566), ('hahaha', 0.05660377358490566), ('hear', 0.03773584905660377), ('speakers', 0.03773584905660377), ('blow', 0.03773584905660377), ('fill', 0.03773584905660377), ('cup', 0.03773584905660377), ('drink', 0.037735

[('share', 1.0), ('crime', 0.5), ('souls', 0.5), ('intertwine', 0.5), ('blind', 0.5), ('watch', 0.5), ('speak', 0.5), ('heart', 0.5), ('limit', 0.16666666666666666), ('sky', 0.16666666666666666), ('fly', 0.16666666666666666), ('amount', 0.16666666666666666), ('tears', 0.16666666666666666), ('eyes', 0.16666666666666666), ('cry', 0.16666666666666666), ('breath', 0.16666666666666666), ('air', 0.16666666666666666), ('promise', 0.16666666666666666), ('climb', 0.16666666666666666), ('mountain', 0.16666666666666666), ('steep', 0.16666666666666666), ('cupid', 0.16666666666666666), ('lie', 0.16666666666666666), ('arrow', 0.16666666666666666), ('miss', 0.16666666666666666), ('love', 0.16666666666666666), ('regret', 0.16666666666666666), ('open', 0.16666666666666666), ('mind', 0.16666666666666666), ('clear', 0.16666666666666666), ('head', 0.16666666666666666), ('gotta', 0.16666666666666666), ('wake', 0.16666666666666666), ('empty', 0.16666666666666666), ('bed', 0.16666666666666666), ('life', 0.16

[('hey', 1.0), ('ooh', 0.7777777777777778), ('touch', 0.5555555555555556), ('blinded', 0.4444444444444444), ('lights', 0.4444444444444444), ('sleep', 0.4444444444444444), ('feel', 0.4444444444444444), ('baby', 0.2222222222222222), ('city', 0.2222222222222222), ('cold', 0.2222222222222222), ('empty', 0.2222222222222222), ('judge', 0.2222222222222222), ('drowning', 0.2222222222222222), ('night', 0.2222222222222222), ('trust', 0.2222222222222222), ('time', 0.2222222222222222), ('phone', 0.2222222222222222), ('tryna', 0.1111111111111111), ('call', 0.1111111111111111), ('long', 0.1111111111111111), ('show', 0.1111111111111111), ('love', 0.1111111111111111), ('withdrawals', 0.1111111111111111), ('turn', 0.1111111111111111), ('running', 0.1111111111111111), ('sun', 0.1111111111111111), ('light', 0.1111111111111111), ('sky', 0.1111111111111111), ('hit', 0.1111111111111111), ('road', 0.1111111111111111), ('overdrive', 0.1111111111111111), ('walking', 0.1111111111111111)]
Push myself score:1
Hav

[('doctor', 1.0), ('back', 0.6111111111111112), ('life', 0.6111111111111112), ('bring', 0.3888888888888889), ('fuckin', 0.2777777777777778), ('fuck', 0.2777777777777778), ('lose', 0.2222222222222222), ('mind', 0.2222222222222222), ('long', 0.2222222222222222), ('runnin', 0.2222222222222222), ('time', 0.2222222222222222), ('call', 0.2222222222222222), ('day', 0.16666666666666666), ('gon', 0.16666666666666666), ('dre', 0.16666666666666666), ('told', 0.1111111111111111), ('play', 0.1111111111111111), ('believe', 0.1111111111111111), ('wack', 0.1111111111111111), ('turn', 0.1111111111111111), ('mic', 0.1111111111111111), ('shit', 0.1111111111111111), ('remember', 0.1111111111111111), ('booth', 0.1111111111111111), ('givin', 0.1111111111111111), ('friends', 0.1111111111111111), ('ass', 0.1111111111111111), ('cracker', 0.1111111111111111), ('world', 0.05555555555555555), ('pay', 0.05555555555555555), ('tape', 0.05555555555555555), ('lay', 0.05555555555555555), ('record', 0.05555555555555555)

[('crazy', 1.0), ('remember', 0.4444444444444444), ('make', 0.3333333333333333), ('lost', 0.1111111111111111), ('mind', 0.1111111111111111), ('pleasant', 0.1111111111111111), ('place', 0.1111111111111111), ('emotions', 0.1111111111111111), ('echo', 0.1111111111111111), ('space', 0.1111111111111111), ('touch', 0.1111111111111111), ('knew', 0.1111111111111111), ('possibly', 0.1111111111111111), ('hope', 0.1111111111111111), ('time', 0.1111111111111111), ('life', 0.1111111111111111), ('advice', 0.1111111111111111), ('bless', 0.1111111111111111), ('soul', 0.1111111111111111), ('control', 0.1111111111111111), ('heroes', 0.1111111111111111), ('heart', 0.1111111111111111), ('lose', 0.1111111111111111), ('lives', 0.1111111111111111), ('limb', 0.1111111111111111), ('looked', 0.1111111111111111), ('fun', 0.1111111111111111), ('coincidence', 0.1111111111111111), ('probably', 0.1111111111111111)]
Push myself score:0
Have fun score:1
Stress relief score:1
Relax / focus score:1
Get motivated score:0

...

[('love', 1.0), ('home', 0.32142857142857145), ('matter', 0.2857142857142857), ('back', 0.14285714285714285), ('pick', 0.14285714285714285), ('feel', 0.07142857142857142), ('pre', 0.07142857142857142), ('moving', 0.07142857142857142), ('lost', 0.07142857142857142), ('middle', 0.07142857142857142), ('things', 0.07142857142857142), ('forget', 0.07142857142857142), ('told', 0.07142857142857142), ('call', 0.07142857142857142), ('give', 0.07142857142857142), ('1', 0.03571428571428571), ('living', 0.03571428571428571), ('fast', 0.03571428571428571), ('lane', 0.03571428571428571), ('time', 0.03571428571428571), ('precious', 0.03571428571428571), ('counting', 0.03571428571428571), ('seconds', 0.03571428571428571), ('skin', 0.03571428571428571), ('direction', 0.03571428571428571), ('sorry', 0.03571428571428571), ('different', 0.03571428571428571), ('tearing', 0.03571428571428571), ('post', 0.03571428571428571), ('2', 0.03571428571428571), ('heaven', 0.03571428571428571), ('hesitation', 0.035714

[('love', 1.0), ('lost', 1.0), ('graffiti', 0.7142857142857143), ('overpass', 0.5714285714285714), ('leave', 0.5714285714285714), ('baby', 0.5714285714285714), ('worth', 0.42857142857142855), ('fade', 0.42857142857142855), ('time', 0.42857142857142855), ('change', 0.42857142857142855), ('remember', 0.42857142857142855), ('full', 0.42857142857142855), ('stop', 0.42857142857142855), ('rain', 0.2857142857142857), ('hip', 0.2857142857142857), ('pre', 0.2857142857142857), ('times', 0.2857142857142857), ('cards', 0.2857142857142857), ('stacked', 0.2857142857142857), ('post', 0.2857142857142857), ('translationsespañolportuguêsdeutsch', 0.14285714285714285), ('1', 0.14285714285714285), ('dark', 0.14285714285714285), ('parade', 0.14285714285714285), ('rough', 0.14285714285714285), ('patch', 0.14285714285714285), ('friends', 0.14285714285714285), ('celebration', 0.14285714285714285), ('hooray', 0.14285714285714285), ('photographs', 0.14285714285714285), ('sepia', 0.14285714285714285), ('tones', 

[('fast', 1.0), ('car', 0.8), ('feeling', 0.6), ('make', 0.4), ('driving', 0.4), ('felt', 0.4), ('city', 0.3), ('live', 0.3), ('working', 0.2), ('pre', 0.2), ('fly', 0.2), ('away', 0.2), ('gotta', 0.2), ('decision', 0.2), ('leave', 0.2), ('tonight', 0.2), ('remember', 0.2), ('speed', 0.2), ('drunk', 0.2), ('lights', 0.2), ('lay', 0.2), ('arm', 0.2), ('nice', 0.2), ('wrapped', 0.2), ('round', 0.2), ('shoulder', 0.2), ('belonged', 0.2), ('body', 0.2), ('work', 0.2), ('woah', 0.1), ('1', 0.1), ('ticket', 0.1), ('deal', 0.1), ('place', 0.1), ('starting', 0.1), ('zero', 0.1), ('lose', 0.1), ('prove', 0.1), ('plan', 0.1), ('convenience', 0.1), ('store', 0.1), ('managed', 0.1), ('save', 0.1), ('bit', 0.1), ('money', 0.1), ('drive', 0.1), ('border', 0.1), ('jobs', 0.1), ('finally', 0.1), ('means', 0.1), ('living', 0.1), ('2', 0.1), ('problem', 0.1), ('lives', 0.1), ('bottle', 0.1), ('young', 0.1), ('mama', 0.1), ('left', 0.1), ('wanted', 0.1), ('life', 0.1), ('give', 0.1), ('quit', 0.1), ('sch

[('geronimo', 1.0), ('bombs', 0.46875), ('away', 0.46875), ('feel', 0.25), ('waterfall', 0.21875), ('love', 0.21875), ('curtains', 0.1875), ('make', 0.15625), ('leap', 0.15625), ('post', 0.09375), ('lost', 0.0625), ('rushed', 0.0625), ('broken', 0.0625), ('1', 0.03125), ('coming', 0.03125), ('back', 0.03125), ('steal', 0.03125), ('gap', 0.03125), ('held', 0.03125), ('hand', 0.03125), ('tossed', 0.03125), ('understand', 0.03125), ('waiting', 0.03125), ('2', 0.03125), ('moving', 0.03125), ('fast', 0.03125), ('crushed', 0.03125), ('past', 0.03125), ('boy', 0.03125), ('toy', 0.03125), ('coy', 0.03125), ('stand', 0.03125), ('found', 0.03125), ('friend', 0.03125), ('falling', 0.03125), ('crashing', 0.03125), ('sound', 0.03125), ('sets', 0.03125), ('free', 0.03125), ('fall', 0.03125), ('knees', 0.03125), ('39embed', 0.03125)]
Push myself score:0
Have fun score:0
Stress relief score:1
Relax / focus score:0
Get motivated score:0


******** Body ********
[('body', 1.0), ('innocence', 0.933333333

TypeError: can only concatenate str (not "float") to str

In [16]:
# Here the new scores are written to the file more_lyric.csv. 
# Either change this to write to lyrics.csv or copy-paste. 
# This was only the test phase so far.
# (Same as with the lyrics before)
df.to_csv('more_lyrics.csv',sep=';',encoding='utf-8-sig')

In [ ]:
print('Push myself songs: '+str(push_myself_counter))
print('Have fun songs: '+str(have_fun_counter))
print('Stress relief songs: '+str(stress_relief_counter))
print('Relax/Focus songs: '+ str(relax_focus_counter))
print('Get motivated songs: '+str(get_motivated_counter))

In [ ]:
#processing for diagram
t_array = df["bpm"].to_numpy() 
print(t_array)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

n, bins, patches = plt.hist(t_array)
plt.show()

In [20]:
print(t_array.size)

56


In [5]:
#Tutorial for Genius
'''from lyricsgenius import Genius
token = 'Wbdl4zQaphgb0HpfLCqJuhJWNU2B2m7NDXTh3oN9TE5bmM77K1fS68czEbS_wSHy'
genius = Genius(token)
artist = genius.search_artist("Andy Shauf", max_songs=3, sort="title")
print(artist.songs)
# Way 1
song = genius.search_song("To You", artist.name)

# Way 2
# this will search artist.songs first
# and if not found, uses search_song
song = artist.song("To You")

print(song.lyrics)'''


Searching for songs by Andy Shauf...

Song 1: "Alexander All Alone"
"All Of My Love" is not valid. Skipping.
Song 2: "All the Same"
Song 3: "Angela"

Reached user-specified song limit (3).
Done. Found 3 songs.
[Song(id, artist, ...), Song(id, artist, ...), Song(id, artist, ...)]
Searching for "To You" by Andy Shauf...
Done.
Searching for "To You" by Andy Shauf...
Done.
To You LyricsJeremy, can we talk a minute?
I've got some things that I need to
Get off of my chest

I know that we've had a few
And it's far too late
But if I wait, I might never tell you
Can we find somewhere quiet?
Let's go outside
I've got some smokes if you've got a light
It's just that sometimes when I'm by your side
It feels so right
It feels like nothing could go wrong
Does it ever
Feel like that to you?
Does it ever
Feel like that to you?
Oh, I don't know what I mean
It sounded wrong and I'm just tired
Not being weird
I just mean that you're a good friend
It's hard to explain
Just forget I said anything
Get over 

In [35]:
#Test for bpm API, does not work
key= '0bf7106cdcb361896dd1cbf071ff715e'
url_bpm = 'https://api.getsongbpm.comsong/?api_key='+key
url_song = "https://api.getsongbpm.com/song/?api_key="+key+"&id=983pB"
response = requests.get(url_song)
#json_data = json.loads(response.content)
print(response)

<Response [403]>
